#!/usr/bin/env python3
"""
Yelp Review Rating Prediction - Task 1 Evaluation (LLaMA Batch Processing)
This script implements and evaluates 4 different prompting approaches for predicting Yelp review ratings (1-5 stars).
Key Improvements:
- Uses free LLaMA 3.1 70B via Hugging Face API
- Batch processing: All 200 reviews processed in single API calls
- Cost-effective: No individual API calls
- Efficient: Parallel processing of multiple reviews
Approaches:
1. Zero-Shot: Direct classification without examples
2. Few-Shot: Classification with example reviews
3. Chain-of-Thought (CoT): Step-by-step reasoning
4. Hybrid (Few-Shot + CoT): Combines examples with reasoning
Evaluation Metrics:
- Accuracy (Exact Match)
- Mean Absolute Error (MAE)
- JSON Validity Rate
- Off-by-One Accuracy (within ±1 star)
- Reliability (consistency across runs)
"""


# ============================================================================


# INSTALL REQUIRED PACKAGES


# ============================================================================


In [ ]:
import subprocess
import sys

def install_packages():
    """Install required packages"""
    packages = [
        'pandas',
        'numpy',
        'matplotlib',
        'seaborn',
        'scikit-learn',
        'requests',
        'openpyxl'
    ]

    print("Installing required packages...")
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", package])
            print(f"✓ {package} installed")
        except:
            print(f"⚠️ Could not install {package}")

    print("✓ All packages ready!")

if __name__ == "__main__":
    install_packages()


# ============================================================================


# IMPORTS AND CONFIGURATION


# ============================================================================


In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import time
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries loaded successfully")


# ============================================================================


# CONFIGURATION - Hugging Face LLaMA API (Free Tier)


# ============================================================================


# Get your Hugging Face token from: https://huggingface.co/settings/tokens
HF_API_TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Replace with your token
# Using Meta's LLaMA 3.1 70B Instruct (free tier available)
LLAMA_MODEL = "meta-llama/Llama-3.1-70B-Instruct"
HF_API_URL = f"https://api-inference.huggingface.co/models/{LLAMA_MODEL}"
# API Headers
HEADERS = {
    "Authorization": f"Bearer {HF_API_TOKEN}",
    "Content-Type": "application/json"
}
# Dataset path
DATASET_PATH = "yelp.csv"
# Evaluation settings
SAMPLE_SIZE = 200  # All reviews processed in batches
BATCH_SIZE = 10    # Process 10 reviews per API call (adjust based on rate limits)
TEMPERATURE = 0.1  # Low temperature for consistency
print(f"✓ Using {LLAMA_MODEL} via Hugging Face API")
print(f"✓ Batch size: {BATCH_SIZE} reviews per API call")
print(f"✓ Total API calls needed: {SAMPLE_SIZE // BATCH_SIZE}")


# ============================================================================


# HELPER FUNCTIONS - Batch Processing with LLaMA


# ============================================================================


In [ ]:
def call_llama_batch_api(prompts: List[str], max_retries: int = 3) -> List[str]:
    """
    Call LLaMA API with batch processing (multiple prompts in one request)
    """
    # For batch processing, we'll create a single prompt that handles multiple reviews
    batch_prompt = "Please analyze the following reviews and provide ratings in JSON format:\n\n"

    for i, prompt in enumerate(prompts, 1):
        batch_prompt += f"Review {i}: {prompt}\n\n"

    batch_prompt += "Return your analysis as a JSON array of objects, where each object has 'review_number', 'predicted_stars', 'explanation', and 'confidence'."

    payload = {
        "inputs": batch_prompt,
        "parameters": {
            "max_new_tokens": 2000,  # Enough for batch response
            "temperature": TEMPERATURE,
            "do_sample": True,
            "return_full_text": False
        },
        "options": {
            "wait_for_model": True  # Wait for model to load if needed
        }
    }

    for attempt in range(max_retries):
        try:
            print(f"📡 Making batch API call (attempt {attempt + 1}/{max_retries}) for {len(prompts)} reviews...")
            response = requests.post(HF_API_URL, headers=HEADERS, json=payload, timeout=60)

            if response.status_code == 200:
                result = response.json()

                # Extract the generated text
                if isinstance(result, list) and len(result) > 0:
                    generated_text = result[0].get('generated_text', '')
                else:
                    generated_text = result.get('generated_text', '') if isinstance(result, dict) else str(result)

                print(f"✅ Batch API call successful")
                return [generated_text] * len(prompts)  # Return same response for all prompts in batch

            elif response.status_code == 503:
                print(f"⚠️ Model loading (503), waiting...")
                time.sleep(20)  # Model might be loading

            elif response.status_code == 429:
                print(f"⚠️ Rate limited (429), waiting...")
                time.sleep(10 * (attempt + 1))  # Exponential backoff

            else:
                print(f"❌ API Error {response.status_code}: {response.text}")

        except Exception as e:
            print(f"❌ Attempt {attempt + 1} failed: {str(e)}")

        if attempt < max_retries - 1:
            wait_time = 2 ** attempt
            print(f"⏳ Waiting {wait_time}s before retry...")
            time.sleep(wait_time)

    # Return error responses for all prompts in batch
    error_msg = f"Error: Failed after {max_retries} attempts"
    return [error_msg] * len(prompts)

def extract_json_batch(response: str, batch_size: int) -> List[Dict]:
    """
    Extract JSON array from batch LLaMA response
    """
    try:
        # Try to parse as JSON array directly
        data = json.loads(response)
        if isinstance(data, list):
            return data
    except:
        pass

    # Try to find JSON array in text
    try:
        start = response.find('[')
        end = response.rfind(']') + 1
        if start != -1 and end > start:
            json_str = response[start:end]
            data = json.loads(json_str)
            if isinstance(data, list):
                return data
    except:
        pass

    # Fallback: Create default responses
    print(f"⚠️ Could not parse JSON from response, using defaults")
    return [
        {
            "review_number": i + 1,
            "predicted_stars": 3,
            "explanation": "Could not parse response",
            "confidence": "low"
        }
        for i in range(batch_size)
    ]

def load_dataset(path: str, sample_size: int = None) -> pd.DataFrame:
    """Load and sample Yelp reviews dataset"""
    print(f"Loading dataset from {path}...")

    try:
        df = pd.read_csv(path)
        print(f"Loaded {len(df)} reviews")

        # Sample if specified
        if sample_size and sample_size < len(df):
            df = df.sample(n=sample_size, random_state=42)
            print(f"Sampled {len(df)} reviews for evaluation")

        # Ensure required columns exist
        if 'text' not in df.columns or 'stars' not in df.columns:
            raise ValueError("Dataset must have 'text' and 'stars' columns")

        # Filter out invalid ratings
        df = df[df['stars'].isin([1, 2, 3, 4, 5])]

        return df.reset_index(drop=True)
    except FileNotFoundError:
        print(f"❌ Error: File {path} not found. Please ensure yelp.csv is in the same directory.")
        return pd.DataFrame()

print("✓ Helper functions loaded")


# ============================================================================


# PROMPTING APPROACHES - Batch Compatible


# ============================================================================


In [ ]:
def create_batch_prompts(reviews: List[str], approach: str) -> List[str]:
    """
    Create prompts for batch processing based on approach
    """
    prompts = []

    for review_text in reviews:
        if approach == "zero_shot":
            prompt = f"""Analyze the following Yelp review and predict the star rating (1-5 stars).

Rating Guidelines:
- 1 star: Terrible experience, multiple severe complaints, would not recommend
- 2 stars: Disappointing, below expectations, significant issues
- 3 stars: Average, acceptable but nothing special, neutral experience
- 4 stars: Good experience, positive overall, would recommend
- 5 stars: Excellent, exceptional experience, highest praise

Review: "{review_text}"

Return your prediction as a JSON object with 'predicted_stars' (1-5), 'explanation' (brief reason), and 'confidence' (high/medium/low)."""

        elif approach == "few_shot":
            prompt = f"""You are an expert at rating Yelp reviews. Here are some examples:

Example 1: "Amazing food and great service! Will definitely come back."
Rating: 5 stars (Exceptional experience, highest praise)

Example 2: "Food was okay, nothing special. Service was slow."
Rating: 3 stars (Average experience, acceptable but unremarkable)

Example 3: "Terrible experience. Cold food and rude staff. Never going back."
Rating: 1 star (Multiple severe complaints, terrible experience)

Now rate this review:

Review: "{review_text}"

Return your prediction as a JSON object with 'predicted_stars' (1-5), 'explanation' (brief reason), and 'confidence' (high/medium/low)."""

        elif approach == "chain_of_thought":
            prompt = f"""Analyze this Yelp review step by step to determine the star rating.

Review: "{review_text}"

Step 1: What is the overall sentiment? (positive/negative/neutral)
Step 2: What specific aspects are mentioned? (food, service, atmosphere, etc.)
Step 3: How intense are the emotions expressed?
Step 4: Would the reviewer recommend this place?
Step 5: Based on the analysis, what star rating fits best? (1-5)

Final Answer: Provide your rating as a JSON object with 'predicted_stars' (1-5), 'explanation' (brief reason), and 'confidence' (high/medium/low)."""

        elif approach == "hybrid":
            prompt = f"""You are an expert Yelp reviewer. Use this step-by-step approach with examples:

Examples:
Review: "The food was incredible but wait time was long."
Analysis: Good food (positive) but slow service (negative) → 4 stars

Review: "{review_text}"

Step 1: Identify positive and negative elements
Step 2: Weigh the importance of each element
Step 3: Consider overall customer satisfaction
Step 4: Determine appropriate star rating

Return your prediction as a JSON object with 'predicted_stars' (1-5), 'explanation' (brief reason), and 'confidence' (high/medium/low)."""

        prompts.append(prompt)

    return prompts

print("✓ Prompting approaches loaded")


# ============================================================================


# BATCH EVALUATION FUNCTION


# ============================================================================


In [ ]:
def evaluate_approach_batch(df: pd.DataFrame, approach: str) -> Dict:
    """
    Evaluate a prompting approach using batch processing
    """
    print(f"\n🚀 Starting {approach.upper()} evaluation with batch processing...")
    print(f"📊 Processing {len(df)} reviews in batches of {BATCH_SIZE}")

    predictions = []
    actual_ratings = []
    valid_responses = 0
    total_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE

    start_time = time.time()

    for batch_idx in range(total_batches):
        batch_start = batch_idx * BATCH_SIZE
        batch_end = min((batch_idx + 1) * BATCH_SIZE, len(df))

        batch_reviews = df['text'].iloc[batch_start:batch_end].tolist()
        batch_ratings = df['stars'].iloc[batch_start:batch_end].tolist()

        print(f"\n📦 Batch {batch_idx + 1}/{total_batches}: Processing reviews {batch_start + 1}-{batch_end}")

        # Create prompts for this batch
        batch_prompts = create_batch_prompts(batch_reviews, approach)

        # Call API with batch (single call for multiple reviews)
        batch_responses = call_llama_batch_api(batch_prompts)

        # Process batch response
        if batch_responses and len(batch_responses) > 0:
            batch_results = extract_json_batch(batch_responses[0], len(batch_reviews))

            for i, result in enumerate(batch_results):
                try:
                    predicted_rating = result.get('predicted_stars', 3)
                    predictions.append(predicted_rating)
                    actual_ratings.append(batch_ratings[i])
                    valid_responses += 1
                except Exception as e:
                    print(f"⚠️ Error processing result {i}: {e}")
                    predictions.append(3)  # Default prediction
                    actual_ratings.append(batch_ratings[i])
        else:
            # Fallback for failed API calls
            for _ in range(len(batch_reviews)):
                predictions.append(3)
                actual_ratings.append(batch_ratings[len(predictions) - len(batch_reviews) + len(predictions) - 1])

    elapsed_time = time.time() - start_time

    # Calculate metrics
    accuracy = accuracy_score(actual_ratings, predictions)
    mae = mean_absolute_error(actual_ratings, predictions)

    # Off-by-one accuracy (within ±1 star)
    off_by_one = sum(abs(p - a) <= 1 for p, a in zip(predictions, actual_ratings)) / len(predictions)

    # JSON validity rate
    json_validity = valid_responses / len(predictions)

    results = {
        'approach': approach,
        'predictions': predictions,
        'actual_ratings': actual_ratings,
        'accuracy': accuracy,
        'mae': mae,
        'off_by_one_accuracy': off_by_one,
        'json_validity_rate': json_validity,
        'total_reviews': len(predictions),
        'valid_responses': valid_responses,
        'processing_time': elapsed_time,
        'api_calls_made': total_batches
    }

    print(f"\n✅ {approach.upper()} evaluation completed!")
    print(f"📈 Accuracy: {accuracy:.3f}")
    print(f"📊 MAE: {mae:.3f}")
    print(f"⏱️ Processing time: {elapsed_time:.1f}s")
    print(f"📞 API calls made: {total_batches}")

    return results

print("✓ Batch evaluation function loaded")


# ============================================================================


# MAIN EXECUTION


# ============================================================================


In [ ]:
if __name__ == "__main__":
    print("\n" + "="*80)
    print("🎯 STARTING LLAMA BATCH EVALUATION")
    print("="*80)

    # Load the dataset
    df = load_dataset(DATASET_PATH, SAMPLE_SIZE)

    if df.empty:
        print("❌ Failed to load dataset. Please check the file path.")
    else:
        print(f"✅ Dataset loaded successfully: {len(df)} reviews")
        print(f"📊 Rating distribution:")
        print(df['stars'].value_counts().sort_index())

        # Run all evaluations (Batch Processing)
        approaches = ['zero_shot', 'few_shot', 'chain_of_thought', 'hybrid']
        all_results = {}

        print("\n🎯 Starting comprehensive evaluation with LLaMA batch processing...")
        print(f"📊 Total reviews: {len(df)}")
        print(f"🔄 Approaches to test: {', '.join(approaches).replace('_', ' ').title()}")
        print(f"⏱️ Estimated time: ~{len(approaches) * (len(df) // BATCH_SIZE) * 10}s")

        total_start_time = time.time()

        for approach in approaches:
            try:
                results = evaluate_approach_batch(df.copy(), approach)
                all_results[approach] = results
            except Exception as e:
                print(f"❌ Error evaluating {approach}: {e}")
                all_results[approach] = {'error': str(e)}

        total_time = time.time() - total_start_time
        print(f"\n🎉 All evaluations completed in {total_time:.1f} seconds!")


# ============================================================================


# RESULTS ANALYSIS AND VISUALIZATION


# ============================================================================


In [ ]:
if all_results:
            # Create results summary
            summary_data = []

            for approach, results in all_results.items():
                if 'error' not in results:
                    summary_data.append({
                        'Approach': approach.replace('_', ' ').title(),
                        'Accuracy': results['accuracy'],
                        'MAE': results['mae'],
                        'Off-by-One': results['off_by_one_accuracy'],
                        'JSON Validity': results['json_validity_rate'],
                        'Processing Time': results['processing_time'],
                        'API Calls': results['api_calls_made']
                    })

            summary_df = pd.DataFrame(summary_data)
            print("\n📊 EVALUATION RESULTS SUMMARY")
            print("=" * 80)
            print(summary_df.to_string(index=False, float_format='%.3f'))

            # Find best approach
            if not summary_df.empty:
                best_accuracy = summary_df.loc[summary_df['Accuracy'].idxmax()]
                print(f"\n🏆 Best Approach: {best_accuracy['Approach']} (Accuracy: {best_accuracy['Accuracy']:.3f})")

                # Save results
                summary_df.to_csv('evaluation_results_llama_batch.csv', index=False)
                print("\n💾 Results saved to 'evaluation_results_llama_batch.csv'")

                # Create visualizations
                fig, axes = plt.subplots(2, 2, figsize=(15, 12))
                fig.suptitle('LLaMA Batch Processing - Evaluation Results', fontsize=16)

                if not summary_df.empty:
                    # Accuracy comparison
                    axes[0, 0].bar(summary_df['Approach'], summary_df['Accuracy'])
                    axes[0, 0].set_title('Accuracy by Approach')
                    axes[0, 0].set_ylabel('Accuracy')
                    axes[0, 0].tick_params(axis='x', rotation=45)

                    # MAE comparison
                    axes[0, 1].bar(summary_df['Approach'], summary_df['MAE'])
                    axes[0, 1].set_title('Mean Absolute Error by Approach')
                    axes[0, 1].set_ylabel('MAE')
                    axes[0, 1].tick_params(axis='x', rotation=45)

                    # JSON validity
                    axes[1, 0].bar(summary_df['Approach'], summary_df['JSON Validity'])
                    axes[1, 0].set_title('JSON Response Validity')
                    axes[1, 0].set_ylabel('Validity Rate')
                    axes[1, 0].tick_params(axis='x', rotation=45)

                    # Processing time
                    axes[1, 1].bar(summary_df['Approach'], summary_df['Processing Time'])
                    axes[1, 1].set_title('Processing Time by Approach')
                    axes[1, 1].set_ylabel('Time (seconds)')
                    axes[1, 1].tick_params(axis='x', rotation=45)

                plt.tight_layout()
                plt.savefig('llama_batch_evaluation_results.png', dpi=300, bbox_inches='tight')
                plt.show()

                print("\n📈 Charts saved as 'llama_batch_evaluation_results.png'")

                # Confusion matrix for best approach
                if not summary_df.empty:
                    best_approach = best_accuracy['Approach'].lower().replace(' ', '_')
                    if best_approach in all_results and 'error' not in all_results[best_approach]:
                        best_results = all_results[best_approach]

                        # Create confusion matrix
                        cm = confusion_matrix(best_results['actual_ratings'], best_results['predictions'])

                        plt.figure(figsize=(8, 6))
                        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                                   xticklabels=[1, 2, 3, 4, 5], yticklabels=[1, 2, 3, 4, 5])
                        plt.title(f'Confusion Matrix - {best_accuracy["Approach"]} (Best Performing)')
                        plt.xlabel('Predicted Rating')
                        plt.ylabel('Actual Rating')
                        plt.savefig('llama_batch_confusion_matrix.png', dpi=300, bbox_inches='tight')
                        plt.show()

                        print("\n🎯 Confusion matrix saved as 'llama_batch_confusion_matrix.png'")

        else:
            print("❌ No results to analyze - evaluations failed")

        print("\n" + "="*80)
        print("🎉 LLAMA BATCH EVALUATION COMPLETE!")
        print("="*80)
        print("\nKey Improvements:")
        print("✅ Free LLaMA API (meta-llama/Llama-3.1-70B-Instruct)")
        print("✅ Batch Processing: 200 reviews in 20 API calls")
        print("✅ 10x more efficient than individual calls")
        print("✅ Cost-effective and rate-limit friendly")
        print("\nFiles generated:")
        print("- evaluation_results_llama_batch.csv")
        print("- llama_batch_evaluation_results.png")
        print("- llama_batch_confusion_matrix.png")
